### Measurement exploratory data analysis

In [ ]:
DEVICES = ['icomox', 'icomox-fast', 'beaglebone', 'beaglebone-old', 'beaglebone-fan']

DEVICE = DEVICES[2]
T_WAVEFORM = 1
T_SEC = T_WAVEFORM
NFFT = 512

In [ ]:
from tabulate import tabulate
from IPython.display import Markdown, HTML
import matplotlib.pylab as plt
from typing import List, Tuple

import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from scipy.signal import butter, lfilter

from tsfel.feature_extraction.features import fundamental_frequency

import os
import os.path
import sys
sys.path.append('../')
from vibrodiagnostics import mafaulda
from vibrodiagnostics import discovery

pd.set_option("display.notebook_repr_html", False)

Import dataset

In [ ]:
def beaglebone_dataset(filenames: List[str]) -> List[Tuple[str, pd.DataFrame]]:
    dataset = []
    g = 9.81
    milivolts = 1800
    resolution = 2 ** 12
    columns = ['x', 'y', 'z']

    for filename in filenames:
        ts = pd.read_csv(filename, delimiter='\t', index_col=False, header=None, names=columns)
        
        # Calculate amplitude in m/s^2 Beaglebone Black ADC and ADXL335 resolution (VIN 1.8V, 12bits)
        for dim in columns:
            ts[dim] = ts[dim] * (milivolts / resolution)  # ADC to mV
            ts[dim] = (ts[dim] / 180) * g                 # mV to m/s^2 (180 mV/g)
            ts[dim] -= ts[dim].mean()

        ts['t'] = ts.index * (1 / Fs)
        ts.set_index('t', inplace=True)
        dataset.append((os.path.basename(filename), ts))

    return dataset


def icomox_dataset(filenames: List[str]) -> List[Tuple[str, pd.DataFrame]]:
    dataset = []
    g = 9.81

    for filename in filenames:
        ts = pd.read_csv(filename, delimiter=',', index_col=False)
        ts = ts.rename(columns={'aX': 'x', 'aY': 'y', 'aZ': 'z'})
        ts['t'] /= 1000
        print('FS:', 1 / (ts['t'] - ts['t'].shift(1)).mean())
        ts.set_index('t', inplace=True)

        # Convert amplitude from g to m/s^2
        for dim in ['x', 'y', 'z']:
            ts[dim] = ts[dim] * g
            ts[dim] -= ts[dim].mean()

        dataset.append((os.path.basename(filename), ts))

    return dataset

In [ ]:
file_list = []

if DEVICE == 'icomox':
    Fs = 400
    DIR = '../../air_conditioning/vnethq/icomox'
    file_list = ['low_power_accel_klima_c1_1.txt', 'low_power_accel_klima_c1_2.txt', 'low_power_accel_klima_c2_1.txt', 'low_power_accel_klima_c2_2.txt']
    #file_list = ['low_power_accel_iCOMOX_2023-11-09_21-58-29.txt']
    file_list = [os.path.join(DIR, filename) for filename in file_list]
    DATASET = icomox_dataset(file_list)

elif DEVICE == 'icomox-fast':
    Fs = 50000
    file_list = ['accel_klima_c1_1.txt', 'accel_klima_c1_2.txt', 'accel_klima_c2_1.txt', 'accel_klima_c2_2.txt']
    DIR = '../../air_conditioning/vnethq/icomox'
    #file_list = ['accel_iCOMOX_2023-11-09_21-58-29.txt']
    file_list = [os.path.join(DIR, filename) for filename in file_list]
    DATASET = icomox_dataset(file_list)

elif DEVICE == 'beaglebone':
    Fs = 2500

    DIR = '../../air_conditioning/vnethq'
    file_list = [
        '1_top.tsv',
        '1_top_1.tsv',
        '1_top_2.tsv',
        '1_side.tsv',
        '1_side_1.tsv',
        '1_side_2.tsv',
        '1_base.tsv',
        '1_base_1.tsv',
        '1_base_2.tsv',
        '1_rackside.tsv',
        '1_rackside_1.tsv',
        '2_top.tsv',
        '2_top_1.tsv',
        '2_top_2.tsv',
        '2_side.tsv',
        '2_side_1.tsv',
        '2_side_2.tsv',
        '2_base.tsv',
        '2_base_1.tsv',
        '2_base_2.tsv',

    ]
    # file_list = ['1_A.tsv', '1_B.tsv', '1_C.tsv']
    # DIR = '../../air_conditioning/vnethq'
    
    # file_list = ['fan_1.tsv', 'fan_2.tsv']
    # DIR = '../../air_conditioning/test'

    file_list = ['ch1.tsv', 'ch2.tsv', 'ch3.tsv', 'ch4.tsv', 'ch5.tsv']
    DIR = '../../air_conditioning/chladnicka'
    
    file_list = [os.path.join(DIR, filename) for filename in file_list]
    DATASET = beaglebone_dataset(file_list)

elif DEVICE == 'beaglebone-old':
    Fs = 1160     # 1160 Hz +/-150 Hz (1020 - 1340 Hz)
    file_list = [
        'excool_digitalis/high_pressure_pump_40percent__1.tsv',
        'excool_digitalis/high_pressure_pump_40percent__2.tsv',
        'excool_digitalis/high_pressure_pump_40percent_speed_up__3.tsv',
        'excool_digitalis/high_pressure_pump_80percent__4.tsv',
        'shc2/compressor_shc2_klima4_bad.tsv',
        'shc2/compressor_shc2_klima5_good.tsv',
        'vertiv_digitalis/compressor_top_1.tsv',
        'vertiv_digitalis/compressor_top_2.tsv',
        'vertiv_digitalis/compressor_top_3.tsv',
        'vertiv_digitalis/compressor_base.tsv',
        'vertiv_digitalis/compressor_side.tsv'
    ]
    DIR = '../../air_conditioning/'
    file_list = [os.path.join(DIR, filename) for filename in file_list]
    DATASET = beaglebone_dataset(file_list)

elif DEVICE == 'beaglebone-fan':
    Fs = 2500
    file_list = [
        'fan/speeds/1_still.tsv',
        'fan/speeds/2_still.tsv',
        'fan/speeds/3_still.tsv',
        'fan/speeds/1_up.tsv',
        'fan/speeds/2_up.tsv',
        'fan/speeds/3_up.tsv',
        'fan/speeds/1_down.tsv',
        'fan/speeds/2_down.tsv',
        'fan/speeds/3_down.tsv',
    ]
    DIR = '../../air_conditioning/'
    file_list = [os.path.join(DIR, filename) for filename in file_list]
    DATASET = beaglebone_dataset(file_list)

In [ ]:
for name, ts in DATASET:
    display(Markdown(f'**{name}**'))
    ts.info()
    print()

In [ ]:
for name, ts in DATASET:
    display(Markdown(f'**{name}**'))
    display(tabulate(ts.describe(), headers='keys', tablefmt='html'))

Time domain waveform

In [ ]:
for name, ts in DATASET:
    display(Markdown(f'**{name}**'))
    axis = ts.columns
    
    ax = ts[axis].plot(figsize=(20, 8), grid=True, subplots=True)
    for i, axname in enumerate(axis):
        ax[i].set_xlabel('Time [s]')
        ax[i].set_ylabel(f'Amplitude ({axname}) [m/s^2]')
    plt.show()               # plt.savefig('waveform.png')

Time domain waveform zoom detail

In [ ]:
for name, ts in DATASET:
    axis = ts.columns
    display(Markdown(f'**{name}**'))
    ax = (ts[axis].iloc[int(T_WAVEFORM*Fs):int(T_WAVEFORM*Fs)+Fs]
                  .plot(figsize=(20, 10), grid=True, subplots=True))
    
    for i, axname in enumerate(axis):
        ax[i].set_xlabel('Time [s]')
        ax[i].set_ylabel(f'Amplitude ({axname}) [m/s^2]')
        plt.show()      # plt.savefig('waveform_zoom.png')

In [ ]:
def spectogram(x):
    fig, ax = plt.subplots(figsize=(15, 4))
    cmap = plt.get_cmap('inferno')
    pxx, freqs, t, im = plt.specgram(
        x, NFFT=NFFT, Fs=Fs,
        detrend='mean',
        mode='magnitude', scale='dB',
        cmap=cmap, vmin=-60
    )
    fig.colorbar(im, aspect=20, pad=0.04)
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Frequency [Hz]')
    mafaulda.resolution_calc(Fs, NFFT)
    return freqs, pxx


def window_idx(t):
    return (Fs * t) // NFFT + 1


def spectrum_slice(freqs, Pxx, t):
    fig, ax = plt.subplots(2, 1, figsize=(20, 8))
    n = window_idx(t)

    dB = 20 * np.log10(Pxx.T[n] / 0.000001)
    ax[0].plot(freqs, dB)      # 1 dB = 1 um/s^2
    ax[0].grid(True)
    ax[0].set_xlabel('Frequency [Hz]')
    ax[0].set_ylabel('Amplitude [dB]')
    
    ax[1].plot(freqs, Pxx.T[n])
    ax[1].grid(True)
    ax[1].set_xlabel('Frequency [Hz]')
    ax[1].set_ylabel('Amplitude [m/s^2]')
    return n


def get_max_frequency(freqs, Pxx, i):
    max_freq = freqs[np.argmax(Pxx.T[i])]
    return max_freq


def get_peaks(freqs, Pxx, i, top=5):
    amplitudes = Pxx.T[i]
    peaks, _ = find_peaks(amplitudes, distance=3)

    fundamental = get_max_frequency(freqs, Pxx, i)
    f_top = freqs[peaks[np.argsort(amplitudes[peaks])]][::-top]
    y_top = np.sort(amplitudes[peaks])[::-top]

    return pd.DataFrame({
        'f': f_top,
        'y': y_top,
        '1x': f_top / fundamental 
    })


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter(order, [lowcut, highcut], fs=fs, btype='band')
    y = lfilter(b, a, data)
    return y


def get_spectrograms(DATASET: List[pd.DataFrame], axis: str) -> list:
    spectrograms = []

    for name, ts in DATASET:
        base_freq = fundamental_frequency(ts[axis], Fs)
        display(Markdown(f'**{name}** *({axis.upper()} axis, Fundamental = {base_freq:.4f} Hz)*'))
        
        freqs, Pxx = spectogram(ts['x'])
        spectrograms.append((name, freqs, Pxx))
        plt.show()          # plt.savefig(f'x_axis_fft_{NFFT}.png')
    
    return spectrograms


def show_spectrogram_detail(spectrograms: list, axis: str, t: float):
    for name, freqs, Pxx in spectrograms:
        display(Markdown(f'**{name}** ({axis.upper()} axis @ {t}s)'))
        i_window = spectrum_slice(freqs, Pxx, t)
        plt.show()           #plt.savefig(f'x_axis_fft_{NFFT}_at_{T_SEC}s.png')


def show_mms_peaks(spectrograms: list, axis: str, t: float):
    for name, freqs, Pxx in spectrograms:
        display(Markdown(f'**{name}** ({axis.upper()} axis @ {t}s)'))
    
        i_window = window_idx(t)
        peaks = discovery.mms_peak_finder(Pxx.T[i_window])
        
        fig, ax = plt.subplots(1, 1, figsize=(15, 3))
        ax.grid(True)
        ax.plot(freqs, Pxx.T[i_window])
        ax.scatter(freqs[peaks], Pxx.T[i_window][peaks], marker='^', color='red')
        ax.set_xlabel('Frequency [Hz]')
        
        plt.show()


def show_harmonic_series(spectrograms: list, axis: str, t: float):
    # https://stackoverflow.com/questions/1982770/changing-the-color-of-an-axis
    for name, freqs, Pxx in spectrograms:
        display(Markdown(f'**{name}** ({axis.upper()} axis @ {t}s)'))
    
        i_window = window_idx(t)
        h_series = discovery.harmonic_series_detection(freqs, Pxx.T[i_window], Fs, NFFT)
    
        # Find best (sum of harmonics' amplitudes in the largest)
        max_harmonic_amp_idx = np.argmax([
            sum([h[1] for h in s]) / len(s)
            for s in h_series
        ])
        best_harmonic_series = pd.DataFrame(
            h_series[max_harmonic_amp_idx],
            columns=['Frequency [Hz]', 'Amplitude [m/s^2]']
        )
        best_harmonic_series.index += 1
        display(tabulate(best_harmonic_series, headers='keys', tablefmt='html'))
    
        # Plot found harmonic series
        fig, ax = plt.subplots(1, 8, figsize=(30, 4))
        for i in range(8):
            s = h_series[i+1]
            if i == max_harmonic_amp_idx:
                ax[i].xaxis.label.set_color('red')
    
            ax[i].plot(freqs, Pxx.T[i_window])
            ax[i].scatter([x[0] for x in s], [x[1] for x in s], marker='^', color='red')
            ax[i].set_xlabel('Frequency [Hz]')
    
        plt.show()

def show_spectra_largest_amplitudes(spectrograms: list, axis: str, t: float):
    for name, freqs, Pxx in spectrograms:
        display(Markdown(f'**{name}** ({axis.upper()} axis @ {t}s)'))

        i_window = window_idx(t)
        x_fundamental = get_max_frequency(freqs, Pxx, i_window)
        peaks = get_peaks(freqs, Pxx, i_window)
        
        display(Markdown(f'- *Fundamental frequency:* {x_fundamental} Hz'))
        display(tabulate(peaks.head(5), headers='keys', tablefmt='html'))

Spectrogram in X axis

In [ ]:
x_spectra = get_spectrograms(DATASET, 'x')

Spectrogram detail in X axis

In [ ]:
show_spectrogram_detail(x_spectra, 'x', T_SEC)

Peaks in frequency spectrum in X axis
- MMS peak finder algorithm

In [ ]:
show_mms_peaks(x_spectra, 'x', T_SEC)

Harmonic series detection in X axis

In [ ]:
# show_harmonic_series(x_spectra, 'x', T_SEC)

In [ ]:
show_spectra_largest_amplitudes(x_spectra, 'x', T_SEC)

Spectrogram in Y axis

In [ ]:
y_spectra = get_spectrograms(DATASET, 'y')

Spectrogram detail in Y axis

In [ ]:
show_spectrogram_detail(y_spectra, 'y', T_SEC)

Peaks in frequency spectrum in Y axis

In [ ]:
show_mms_peaks(y_spectra, 'y', T_SEC)

Harmonic series detection in Y axis

In [ ]:
# show_harmonic_series(y_spectra, 'y', T_SEC)

In [ ]:
show_spectra_largest_amplitudes(y_spectra, 'y', T_SEC)

Spectrogram in Z axis

In [ ]:
z_spectra = get_spectrograms(DATASET, 'z')

Spectrogram detail in Z axis

In [ ]:
show_spectrogram_detail(z_spectra, 'z', T_SEC)

Peaks in frequency spectrum in Z axis

In [ ]:
show_mms_peaks(z_spectra, 'z', T_SEC)

Harmonic series detection in Z axis

In [ ]:
# show_harmonic_series(z_spectra, 'z', T_SEC)

In [ ]:
show_spectra_largest_amplitudes(z_spectra, 'z', T_SEC)

Histogram

In [ ]:
axis = ['x', 'y', 'z']
for name, ts in DATASET:
    display(Markdown(f'**{name}**'))
    ts[axis].hist(figsize=(10, 5), grid=True, bins=50)
    plt.show()

Orbitals of all cross sections

In [ ]:
for name, ts in DATASET:
    display(Markdown(f'**{name}**'))
    fig, ax = plt.subplots(1, 3, figsize=(20, 4))

    for i, col in enumerate([('x', 'y'), ('x', 'z'), ('y', 'z')]):
        ax[i].scatter(ts[col[0]], ts[col[1]], s=1)
        ax[i].grid(True)
        ax[i].set_xlabel(col[0].upper())
        ax[i].set_ylabel(col[1].upper())
        ax[i].grid(True)
    plt.show()       # plt.savefig('orbitals.png')

Orbitals of 1x harmonic frequency

In [ ]:
x_spectra_by_name = {spec[0]: spec for spec in x_spectra}
y_spectra_by_name = {spec[0]: spec for spec in y_spectra}
z_spectra_by_name = {spec[0]: spec for spec in z_spectra}
t = 5
space = 5

for name, ts in DATASET:
    display(Markdown(f'**{name}**'))
    fig, ax = plt.subplots(1, 3, figsize=(20, 4))

    name, freqs, Pxx = x_spectra_by_name[name]
    x_fundamental = get_max_frequency(freqs, Pxx, window_idx(t))

    name, freqs, Pxx = y_spectra_by_name[name]
    y_fundamental = get_max_frequency(freqs, Pxx, window_idx(t))

    name, freqs, Pxx = z_spectra_by_name[name]
    z_fundamental = get_max_frequency(freqs, Pxx, window_idx(t))

    ts['x_1x'] = butter_bandpass_filter(ts['x'], x_fundamental - space, x_fundamental + space, Fs)
    ts['y_1x'] = butter_bandpass_filter(ts['y'], y_fundamental - space, y_fundamental + space, Fs)
    ts['z_1x'] = butter_bandpass_filter(ts['z'], z_fundamental - space, z_fundamental + space, Fs)
    
    for i, col in enumerate([('x_1x', 'y_1x'), ('x_1x', 'z_1x'), ('y_1x', 'z_1x')]):
        ax[i].scatter(ts[col[0]], ts[col[1]], s=1)
        ax[i].grid(True)
        ax[i].set_xlabel(col[0].upper())
        ax[i].set_ylabel(col[1].upper())
        ax[i].grid(True)
    
    plt.show()       # plt.savefig('orbitals_1x.png')